In [1]:
import pandas as pd
import numpy as np

In [3]:
credits=pd.read_csv(r'C:\Users\Akshaya B\Documents\GitHubProjects\Movie_Recommendation_System\data\credits.csv')
movies=pd.read_csv(r'C:\Users\Akshaya B\Documents\GitHubProjects\Movie_Recommendation_System\data\movies.csv')

In [4]:
movies.head()
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [8]:
movies.shape

(4803, 20)

In [10]:
credits.shape

(4803, 4)

In [13]:
movies=movies.merge(credits,on='title')
movies=movies[['id','title','overview','genres','keywords','cast','crew']]

In [14]:
movies.isnull().sum()
movies.dropna(inplace=True)
movies.duplicated().sum()

np.int64(36)

In [15]:
import ast
def convert(text):
    return [i['name'] for i in ast.literal_eval(text)]


In [16]:
movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)

In [17]:
def convert_cast(text):
    L=[]
    for i in ast.literal_eval(text):
        if len(L)<3:
            L.append(i['name'])
    return L
movies['cast']=movies['cast'].apply(convert_cast)        

In [18]:
def fetch_director(text):
    for i in ast.literal_eval(text):
        if i['job']=='Director':
            return [i['name']]
    return [] 
movies['crew']=movies['crew'].apply(fetch_director)   

In [19]:
movies['overview']=movies['overview'].apply(lambda x:x.split())


In [20]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [21]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [22]:
new_df=movies[['id','title','tags']]

In [23]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\Akshaya B\AppData\Local\Temp\ipykernel_19108\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [24]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\Akshaya B\AppData\Local\Temp\ipykernel_19108\4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [27]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 372.9 kB/s eta 0:00:03
   ------------- -------------------------- 0.5/1.5 MB 372.9 kB/s eta 0:00:03
   -------------------- ------------------- 0.8/1.5 MB 466.0 kB/s eta 0:00:02
   -------------------- ------------------- 0.8/1.5 MB 466.0 kB/s eta 0:00:02
   --------------------------- ------------ 1.0/1.5 MB 518.8 kB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 518.8 kB/s eta 0:00:01
   ---------------------------------- --

In [28]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Akshaya
[nltk_data]     B\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [29]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [30]:
def stem(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [31]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\Akshaya B\AppData\Local\Temp\ipykernel_19108\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [33]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [34]:
from sklearn.metrics.pairwise import cosine_similarity 
similarity=cosine_similarity(vectors)

In [35]:
similarity.shape

(4842, 4842)

In [36]:
def recommend(movie):
    movie=movie.lower()
    if movie not in new_df['title'].str.lower().values:
        print("Movie not found in database")
        return 
    index=new_df[new_df['title'].str.lower()==movie].index[0]
    distances=similarity[index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    print("Recommended Movies:")
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
    

In [37]:
recommend('Avatar')

Recommended Movies:
Aliens vs Predator: Requiem
Falcon Rising
Independence Day
Titan A.E.
Aliens


In [38]:
recommend('Inception')

Recommended Movies:
12 Rounds
Abduction
RED
Krrish
The Animal


In [39]:
recommend('Interstellar')

Recommended Movies:
Guardians of the Galaxy
Silent Running
Space Cowboys
The Martian
Apollo 13


In [40]:
recommend('Silent Running')

Recommended Movies:
Slums of Beverly Hills
H.
Reality Show
This Christmas
The Dead Girl


In [41]:
recommend('Rush Hour')

Movie not found in database


In [42]:
recommend('The Godfather')

Recommended Movies:
Desert Dancer
Step Up
Take the Lead
Footloose
Step Up 2: The Streets


In this project, we built a content-based movie recommendation system using NLP techniques.

The system succesfully recommends movies based on textual similarity and can be extended into a real-world application.